# Building the tools

In [8]:
import arxiv
import os
import re

In [9]:
def search_arxiv(query: str, subject: str = None, topic: str = None, max_results: int = 20):
    """
    Search arXiv for papers.
    
    Args:
        query: Search query string
        subject: Subject area (e.g., 'Artificial Intelligence')
        topic: Topic within subject (e.g., 'Healthcare', 'NLP')
        max_results: Maximum number of results to return
    """
    # Build enhanced query with subject/topic if provided
    full_query = query
    if subject:
        full_query = f"{subject} {full_query}"
    if topic:
        full_query = f"{topic} {full_query}"
    
    search = arxiv.Search(
        query=full_query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance,
    )
    results = []
    for result in search.results():
        results.append({
            "source": "arxiv",
            "id": result.entry_id,
            "title": result.title,
            "abstract": result.summary,
            "year": result.published.year,
            "venue": "arXiv",
            "authors": [a.name for a in result.authors],
            "citations": None,
            "pdf_url": result.pdf_url,
            "landing_url": result.entry_id,
            "subject": subject,
            "topic": topic,
        })
    return results

In [21]:
import requests
from pathlib import Path
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# Max filename length - reduced to ensure total path stays under Windows 260 char limit
# Path structure: base (~150) + /subject (~50) + /topic (~50) + /filename + .pdf
# So filename should be ~100 max to stay safe
MAX_FILENAME_LENGTH = 100

# RAG SETUP paths (relative to Tools Server/RAG SETUP)
RAG_SETUP_PATH = Path(__file__).resolve().parent.parent.parent / "Tools Server" / "RAG SETUP" if "__file__" in dir() else Path(r"c:\Users\User\Desktop\llms\Project\Research Assistant Multi Agent System\Tools Server\RAG SETUP")
PAPERS_PATH = RAG_SETUP_PATH / "Papers"
VECTORDB_PATH = RAG_SETUP_PATH / "VectorDB"


def sanitize_filename(name: str, max_length: int = MAX_FILENAME_LENGTH) -> str:
    """Remove invalid characters from filename and limit length."""
    # Remove characters not allowed in Windows filenames: \ / : * ? " < > |
    sanitized = re.sub(r'[\\/:*?"<>|]', '', name)
    # Replace multiple spaces with single space
    sanitized = re.sub(r'\s+', ' ', sanitized)
    # Trim and limit length
    return sanitized.strip()[:max_length]


def shorten_title(title: str, max_length: int = MAX_FILENAME_LENGTH) -> str:
    """
    Shorten a paper title intelligently.
    - If ≤ max_length: return as-is
    - If > max_length and has ':': use part before ':'
    - Otherwise: crop at max_length
    """
    if len(title) <= max_length:
        return title
    
    # Try to use the main title (before the colon)
    if ':' in title:
        main_title = title.split(':')[0].strip()
        if len(main_title) <= max_length and len(main_title) > 10:  # Ensure it's meaningful
            return main_title
    
    # Fallback: crop at max_length
    return title[:max_length].strip()


def _extract_paper_metadata(file_path: Path, base_path: Path) -> dict:
    """
    Extract metadata from a paper's file path.
    Expected structure: base_path/Subject/Topic/title - year.pdf
    """
    metadata = {
        "subject": "Unknown",
        "topic": "Unknown", 
        "paper_title": file_path.stem,
        "year": None,
        "file_name": file_path.name,
    }
    
    try:
        # Get relative path parts
        rel_path = file_path.relative_to(base_path)
        parts = rel_path.parts
        
        if len(parts) >= 3:
            metadata["subject"] = parts[0]
            metadata["topic"] = parts[1]
        elif len(parts) == 2:
            metadata["subject"] = parts[0]
            
        # Extract year from filename: "title - year.pdf"
        stem = file_path.stem
        if " - " in stem:
            title_part, year_part = stem.rsplit(" - ", 1)
            metadata["paper_title"] = title_part.strip()
            try:
                metadata["year"] = int(year_part.strip())
            except ValueError:
                pass
                
    except Exception:
        pass
    
    return metadata


def _add_to_vectordb(file_path: Path, papers_base_path: Path = PAPERS_PATH, vectordb_path: Path = VECTORDB_PATH) -> bool:
    """
    Add a downloaded PDF to the vector database.
    
    Args:
        file_path: Path to the downloaded PDF
        papers_base_path: Base path of the Papers directory
        vectordb_path: Path to the Chroma vector database
        
    Returns:
        True if successfully added, False otherwise
    """
    try:
        # Load the PDF
        loader = PyMuPDFLoader(str(file_path))
        raw_docs = loader.load()
        
        # Extract and add metadata
        paper_metadata = _extract_paper_metadata(file_path, papers_base_path)
        for doc in raw_docs:
            doc.metadata.update(paper_metadata)
            doc.metadata.update({
                "doc_id": file_path.stem,
                "relpath": str(file_path.relative_to(papers_base_path)) if papers_base_path in file_path.parents or file_path.parent == papers_base_path else file_path.name
            })
        
        # Split into chunks
        splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=250)
        split_docs = splitter.split_documents(raw_docs)
        
        # Load existing vectordb or create new one
        embeddings = OpenAIEmbeddings()
        
        if (vectordb_path / "chroma.sqlite3").exists():
            # Add to existing vectordb
            vectordb = Chroma(
                collection_name="research_papers",
                embedding_function=embeddings,
                persist_directory=str(vectordb_path),
            )
            vectordb.add_documents(split_docs)
        else:
            # Create new vectordb
            vectordb_path.mkdir(parents=True, exist_ok=True)
            Chroma.from_documents(
                documents=split_docs,
                embedding=embeddings,
                collection_name="research_papers",
                persist_directory=str(vectordb_path),
            )
        
        return True
    except Exception as e:
        print(f"Warning: Failed to add document to vectordb: {e}")
        return False


def download_pdf(paper: dict, save_dir: str = None, file_name: str = None, add_to_vectordb: bool = True) -> str:
    """
    Download PDF for a paper to: Papers/subject/topic/filename.pdf
    and optionally add it to the vector database.
    
    Args:
        paper: Paper dict with pdf_url, title, subject, topic, year
        save_dir: Base directory for downloads (defaults to RAG Papers folder)
        file_name: Optional custom filename (agent can provide a summarized name)
        add_to_vectordb: Whether to automatically add the paper to the vector DB
    
    Returns:
        Path to downloaded file (and status of vectordb addition)
    """
    pdf_url = paper.get("pdf_url")
    if not pdf_url:
        raise ValueError("No PDF URL available for this paper.")
    
    # Default save_dir to the RAG Papers folder
    if save_dir is None:
        save_dir = PAPERS_PATH
    else:
        save_dir = Path(save_dir)
    
    # Build path: save_dir/subject/topic/
    subject = paper.get("subject") or "General"
    topic = paper.get("topic") or "Uncategorized"
    
    # Sanitize directory names
    subject_dir = sanitize_filename(subject, max_length=50)
    topic_dir = sanitize_filename(topic, max_length=50)
    
    # Use pathlib for cross-platform path handling
    full_dir = Path(save_dir) / subject_dir / topic_dir
    full_dir.mkdir(parents=True, exist_ok=True)
    
    # Determine filename: use provided name, or shorten the title
    if file_name:
        base_name = sanitize_filename(file_name)
    else:
        shortened = shorten_title(paper.get("title", "paper"))
        base_name = sanitize_filename(shortened)
    
    # Add year and .pdf extension
    year = paper.get("year", "")
    filename = f"{base_name} - {year}.pdf" if year else f"{base_name}.pdf"
    
    file_path = full_dir / filename

    resp = requests.get(pdf_url, timeout=60)
    resp.raise_for_status()
    file_path.write_bytes(resp.content)
    
    # Add to vector database if requested
    result_msg = f"Downloaded: {file_path}"
    if add_to_vectordb:
        vectordb_added = _add_to_vectordb(file_path, papers_base_path=Path(save_dir))
        if vectordb_added:
            result_msg += " | Added to vector database ✓"
        else:
            result_msg += " | Failed to add to vector database"

    return result_msg

# Trying out the functions

In [4]:
# Test arXiv search with subject and topic
query = "medical diagnosis deep learning"
subject = "Artificial Intelligence"
topic = "Healthcare"

print(f"Searching arXiv for: {query}")
print(f"Subject: {subject} | Topic: {topic}\n")

results = search_arxiv(query, subject=subject, topic=topic, max_results=5)
print(f"Found {len(results)} papers\n")

for i, paper in enumerate(results, 1):
    print(f"{i}. {paper['title']}")
    print(f"   Year: {paper['year']} | Authors: {', '.join(paper['authors'][:2])}{'...' if len(paper['authors']) > 2 else ''}")
    print()

Searching arXiv for: medical diagnosis deep learning
Subject: Artificial Intelligence | Topic: Healthcare



C:\Users\User\AppData\Local\Temp\ipykernel_28248\1971025978.py:24: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Found 5 papers

1. Medical Knowledge-Guided Deep Curriculum Learning for Elbow Fracture Diagnosis from X-Ray Images
   Year: 2021 | Authors: Jun Luo, Gene Kitamura...

2. A Review on Explainable Artificial Intelligence for Healthcare: Why, How, and When?
   Year: 2023 | Authors: Subrato Bharati, M. Rubaiyat Hossain Mondal...

3. The Artificial Scientist: Logicist, Emergentist, and Universalist Approaches to Artificial General Intelligence
   Year: 2021 | Authors: Michael Timothy Bennett, Yoshihiro Maruyama

4. Privacy-preserving machine learning for healthcare: open challenges and future perspectives
   Year: 2023 | Authors: Alejandro Guerra-Manzanares, L. Julian Lechuga Lopez...

5. Artificial Intelligence Framework for Simulating Clinical Decision-Making: A Markov Decision Process Approach
   Year: 2013 | Authors: Casey C. Bennett, Kris Hauser



In [5]:
# Test the shorten_title function with examples
print("Testing shorten_title():\n")

test_titles = [
    "Short Title",  # ≤ 80
    "A Review on Explainable Artificial Intelligence for Healthcare: Why, How, and When?",  # Has :
    "Medical Knowledge-Guided Deep Curriculum Learning for Elbow Fracture Diagnosis from X-Ray Images",  # No : but long
]

for title in test_titles:
    shortened = shorten_title(title)
    print(f"Original ({len(title)} chars): {title}")
    print(f"Shortened ({len(shortened)} chars): {shortened}\n")

Testing shorten_title():

Original (11 chars): Short Title
Shortened (11 chars): Short Title

Original (83 chars): A Review on Explainable Artificial Intelligence for Healthcare: Why, How, and When?
Shortened (83 chars): A Review on Explainable Artificial Intelligence for Healthcare: Why, How, and When?

Original (96 chars): Medical Knowledge-Guided Deep Curriculum Learning for Elbow Fracture Diagnosis from X-Ray Images
Shortened (96 chars): Medical Knowledge-Guided Deep Curriculum Learning for Elbow Fracture Diagnosis from X-Ray Images



In [6]:
# Test downloading papers
for paper in results[:2]:
    print(f"Title: {paper['title'][:60]}...")
    print(f"Shortened: {shorten_title(paper['title'])}")
    try:
        path = download_pdf(paper, save_dir="test_downloads")
        print(f"✓ Saved to: {path}\n")
    except Exception as e:
        print(f"✗ Failed: {e}\n")

Title: Medical Knowledge-Guided Deep Curriculum Learning for Elbow ...
Shortened: Medical Knowledge-Guided Deep Curriculum Learning for Elbow Fracture Diagnosis from X-Ray Images
✗ Failed: [Errno 2] No such file or directory: 'test_downloads\\Artificial Intelligence\\Healthcare\\Medical Knowledge-Guided Deep Curriculum Learning for Elbow Fracture Diagnosis from X-Ray Images - 2021.pdf'

Title: A Review on Explainable Artificial Intelligence for Healthca...
Shortened: A Review on Explainable Artificial Intelligence for Healthcare: Why, How, and When?
✗ Failed: [Errno 2] No such file or directory: 'test_downloads\\Artificial Intelligence\\Healthcare\\Medical Knowledge-Guided Deep Curriculum Learning for Elbow Fracture Diagnosis from X-Ray Images - 2021.pdf'

Title: A Review on Explainable Artificial Intelligence for Healthca...
Shortened: A Review on Explainable Artificial Intelligence for Healthcare: Why, How, and When?
✓ Saved to: test_downloads\Artificial Intelligence\Healthcare\A Rev

# Building the Agent

In [11]:
import operator
from typing import List, Annotated, TypedDict
from langchain_core.messages import AnyMessage, SystemMessage, ToolMessage, AIMessage
from langgraph.graph import StateGraph, START, END


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        want_tools = isinstance(result, AIMessage) and bool(getattr(result, "tool_calls", None))
        return  want_tools

    async def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = await self.model.ainvoke(messages) # aynchronous invoke
        return {'messages': [message]}

    async def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = await self.tools[t['name']].ainvoke(t['args']) # aynchronous invoke
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

# Trying out the agents with its tools

In [22]:
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field
from typing import Optional

# Define input schemas for the tools
class SearchArxivInput(BaseModel):
    query: str = Field(..., description="Search query for finding papers")
    subject: Optional[str] = Field(default=None, description="Subject area (e.g., 'Artificial Intelligence')")
    topic: Optional[str] = Field(default=None, description="Topic within subject (e.g., 'Healthcare', 'NLP')")
    max_results: int = Field(default=10, description="Maximum number of results to return")

class DownloadPdfInput(BaseModel):
    pdf_url: str = Field(..., description="The PDF URL from the search results")
    title: str = Field(..., description="Title of the paper")
    year: Optional[int] = Field(default=None, description="Publication year")
    subject: Optional[str] = Field(default=None, description="Subject area for organizing the download")
    topic: Optional[str] = Field(default=None, description="Topic for organizing the download")
    add_to_vectordb: bool = Field(default=True, description="Whether to add the paper to the RAG vector database")

# Wrapper function that converts flat args to paper dict
def download_pdf_wrapper(pdf_url: str, title: str, year: int = None, subject: str = None, topic: str = None, add_to_vectordb: bool = True) -> str:
    paper = {
        "pdf_url": pdf_url,
        "title": title,
        "year": year,
        "subject": subject,
        "topic": topic,
    }
    return download_pdf(paper, add_to_vectordb=add_to_vectordb)

# Create structured tools
search_tool = StructuredTool.from_function(
    name="search_arxiv",
    description="Search arXiv for academic papers. Returns a list of papers with title, abstract, authors, year, pdf_url, subject, and topic.",
    func=search_arxiv,
    args_schema=SearchArxivInput,
)

download_tool = StructuredTool.from_function(
    name="download_pdf",
    description="Download a PDF paper and automatically add it to the RAG vector database. Use the pdf_url and title from search results. Saves to: Papers/subject/topic/title - year.pdf",
    func=download_pdf_wrapper,
    args_schema=DownloadPdfInput,
)

tools = [search_tool, download_tool]

# Initialize the agent
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

system_prompt = """You are a research assistant that helps users find and download academic papers from arXiv.

You have two tools:
1. search_arxiv: Search for papers on arXiv by query, subject, and topic
2. download_pdf: Download a paper's PDF using its pdf_url and title from search results. 
   The paper will be automatically added to the RAG vector database for future retrieval.

When a user asks you to find papers:
- Use search_arxiv with relevant query terms
- Include subject and topic if the user specifies them

When a user asks to download a paper:
- Extract the pdf_url, title, year, subject, and topic from the search results
- Pass these to download_pdf
- The PDF will be saved to: Papers/subject/topic/title - year.pdf
- The paper will automatically be indexed in the vector database

Always summarize the results for the user."""

agent = Agent(llm, tools, system=system_prompt)
print("Agent initialized with tools:", [t.name for t in tools])

Agent initialized with tools: ['search_arxiv', 'download_pdf']


In [13]:
from langchain_core.messages import HumanMessage

# Test the agent
async def test_agent():
    query = "Find me 3 papers about deep learning for medical image analysis"
    result = await agent.graph.ainvoke({"messages": [HumanMessage(content=query)]})
    return result["messages"][-1].content

# Run the test
import asyncio
response = await test_agent()
print(response)

Calling: {'name': 'search_arxiv', 'args': {'query': 'deep learning medical image analysis', 'max_results': 3}, 'id': 'call_StDAOMTHI44phNpVtuaMbwfm', 'type': 'tool_call'}


C:\Users\User\AppData\Local\Temp\ipykernel_28248\1971025978.py:24: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Back to the model!
Here are three papers related to deep learning for medical image analysis:

1. **[A Survey on Active Learning and Human-in-the-Loop Deep Learning for Medical Image Analysis](http://arxiv.org/abs/1910.02923v2)** (2019)
   - **Authors**: Samuel Budd, Emma C Robinson, Bernhard Kainz
   - **Abstract**: This paper reviews the role of human involvement in deep learning systems for medical image analysis. It discusses techniques for active learning, interaction with model outputs, practical considerations for deployment, and future research directions.
   - **[Download PDF](https://arxiv.org/pdf/1910.02923v2)**

2. **[TransMorph: Transformer for unsupervised medical image registration](http://arxiv.org/abs/2111.10480v6)** (2021)
   - **Authors**: Junyu Chen, Eric C. Frey, Yufan He, William P. Segars, Ye Li, Yong Du
   - **Abstract**: This paper presents TransMorph, a hybrid Transformer-ConvNet model for volumetric medical image registration. It evaluates the model's perform

In [23]:
# Test downloading a paper - it should save to Papers folder and add to vectordb
query = "Search for a paper about transformer architecture and download the first result. Use subject 'Artificial Intelligence' and topic 'Deep Learning'"
result = await agent.graph.ainvoke({"messages": [HumanMessage(content=query)]})
print(result["messages"][-1].content)

Calling: {'name': 'search_arxiv', 'args': {'query': 'transformer architecture', 'subject': 'Artificial Intelligence', 'topic': 'Deep Learning', 'max_results': 1}, 'id': 'call_o1vuSVJPR7J9BJkePjEhNuJR', 'type': 'tool_call'}


C:\Users\User\AppData\Local\Temp\ipykernel_28248\1971025978.py:24: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Back to the model!
Calling: {'name': 'download_pdf', 'args': {'pdf_url': 'https://arxiv.org/pdf/2110.01831v1', 'title': 'The Artificial Scientist: Logicist, Emergentist, and Universalist Approaches to Artificial General Intelligence', 'year': 2021, 'subject': 'Artificial Intelligence', 'topic': 'Deep Learning'}, 'id': 'call_Viq24nnqM0stb1wZAmefw6V6', 'type': 'tool_call'}
Calling: {'name': 'download_pdf', 'args': {'pdf_url': 'https://arxiv.org/pdf/2110.01831v1', 'title': 'The Artificial Scientist: Logicist, Emergentist, and Universalist Approaches to Artificial General Intelligence', 'year': 2021, 'subject': 'Artificial Intelligence', 'topic': 'Deep Learning'}, 'id': 'call_Viq24nnqM0stb1wZAmefw6V6', 'type': 'tool_call'}


C:\Users\User\AppData\Local\Temp\ipykernel_28248\1444543260.py:124: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


Back to the model!
I found a paper titled **"The Artificial Scientist: Logicist, Emergentist, and Universalist Approaches to Artificial General Intelligence"** by Michael Timothy Bennett and Yoshihiro Maruyama, published in 2021. 

The paper explores various approaches to artificial general intelligence (AGI) and argues for a unified or hybrid approach to constructing an Artificial Scientist.

I have downloaded the paper, and it is saved as **"The Artificial Scientist - 2021.pdf"** in the directory for Artificial Intelligence and Deep Learning. The paper has also been added to the vector database for future retrieval.
I found a paper titled **"The Artificial Scientist: Logicist, Emergentist, and Universalist Approaches to Artificial General Intelligence"** by Michael Timothy Bennett and Yoshihiro Maruyama, published in 2021. 

The paper explores various approaches to artificial general intelligence (AGI) and argues for a unified or hybrid approach to constructing an Artificial Scientis

In [24]:
# Verify the paper was indexed in the vectordb
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

vectordb = Chroma(
    collection_name="research_papers",
    embedding_function=OpenAIEmbeddings(),
    persist_directory=str(VECTORDB_PATH),
)

# Search for content from the newly downloaded paper
results = vectordb.similarity_search("Artificial Scientist AGI approaches", k=3)
print(f"Found {len(results)} results:")
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(f"Subject: {doc.metadata.get('subject')}")
    print(f"Topic: {doc.metadata.get('topic')}")
    print(f"Paper Title: {doc.metadata.get('paper_title')}")
    print(f"Year: {doc.metadata.get('year')}")
    print(f"Content preview: {doc.page_content[:200]}...")

Found 3 results:

--- Result 1 ---
Subject: Artificial Intelligence
Topic: Deep Learning
Paper Title: The Artificial Scientist
Year: 2021
Content preview: arXiv:2110.01831v1  [cs.AI]  5 Oct 2021
The Artiﬁcial Scientist:
Logicist, Emergentist, and Universalist
Approaches to Artiﬁcial General Intelligence⋆
Michael Timothy Bennett and Yoshihiro Maruyama
Sc...

--- Result 2 ---
Subject: Artificial Intelligence
Topic: Deep Learning
Paper Title: The Artificial Scientist
Year: 2021
Content preview: ded to function within the conﬁnes of a speciﬁc environment, enacted through
what an organism does and, ﬁnally, extending into that environment to store
and retrieve information. All of this seems obv...

--- Result 3 ---
Subject: Artificial Intelligence
Topic: Deep Learning
Paper Title: The Artificial Scientist
Year: 2021
Content preview: The Artiﬁcial Scientist
5
theoretical agent, named AIXI, has been proven to perform such that there is
no other agent which outperforms it in one environment tha

In [ ]:
# Direct test of the download function

# Verify paths are correct
print(f"Papers Path: {PAPERS_PATH}")
print(f"VectorDB Path: {VECTORDB_PATH}")
print(f"Papers Path exists: {PAPERS_PATH.exists()}")
print(f"VectorDB Path exists: {VECTORDB_PATH.exists()}")

# Test paper data (from a real arXiv paper)
test_paper = {
    "pdf_url": "https://arxiv.org/pdf/1706.03762",  # "Attention Is All You Need"
    "title": "Attention Is All You Need",
    "year": 2017,
    "subject": "Artificial Intelligence",
    "topic": "Deep Learning",
}

print(f"\nDownloading test paper: {test_paper['title']}")
result = download_pdf(test_paper, add_to_vectordb=True)
print(f"\nResult: {result}")

Papers Path: c:\Users\User\Desktop\llms\Project\Research Assistant Multi Agent System\Tools Server\RAG SETUP\Papers
VectorDB Path: c:\Users\User\Desktop\llms\Project\Research Assistant Multi Agent System\Tools Server\RAG SETUP\VectorDB
Papers Path exists: True
VectorDB Path exists: True


Result: Downloaded: c:\Users\User\Desktop\llms\Project\Research Assistant Multi Agent System\Tools Server\RAG SETUP\Papers\Artificial Intelligence\Deep Learning\Attention Is All You Need - 2017.pdf | Added to vector database ✓

Result: Downloaded: c:\Users\User\Desktop\llms\Project\Research Assistant Multi Agent System\Tools Server\RAG SETUP\Papers\Artificial Intelligence\Deep Learning\Attention Is All You Need - 2017.pdf | Added to vector database ✓
